# 05 Comparative Visualization

This notebook generates scatter plots and other visualizations, comparing various dependency size metrics and linguistic factors. It uses the standardized plotting style (connecting lines to labels) for all scatter plots.

In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
from importlib import reload

# Custom modules
import data_utils
import plotting

# Reload plotting to ensure latest style
reload(plotting)

%matplotlib inline

# Configuration
DATA_DIR = "data"
PLOTS_DIR = "plots"

## 1. Load Data

In [2]:
# Load metadata and metrics
metadata = data_utils.load_metadata(os.path.join(DATA_DIR, 'metadata.pkl'))
langNames = metadata['langNames']
langnameGroup = metadata['langnameGroup']
group_to_color = metadata['appearance_dict']

with open(os.path.join(DATA_DIR, 'all_langs_average_sizes_filtered.pkl'), 'rb') as f:
    all_langs_average_sizes_filtered = pickle.load(f)

# Load Pre-computed factors (e.g. HCS)
if os.path.exists(os.path.join(DATA_DIR, 'hcs_factors.csv')):
    hcs_df = pd.read_csv(os.path.join(DATA_DIR, 'hcs_factors.csv'))
else:
    print("Warning: hcs_factors.csv not found. Please run 04_data_processing using compute_factors.")
    hcs_df = pd.DataFrame()

if os.path.exists(os.path.join(DATA_DIR, 'vo_vs_hi_scores.csv')):
    vo_hi_df = pd.read_csv(os.path.join(DATA_DIR, 'vo_vs_hi_scores.csv'))
else:
    print("Warning: vo_vs_hi_scores.csv not found.")
    vo_hi_df = pd.DataFrame()

print(f"Loaded data for {len(all_langs_average_sizes_filtered)} languages")

Loaded metadata from data/metadata.pkl
Loaded data for 185 languages


## 2. MAL Plots (Mean Average Length)

In [3]:
# Example MAL plot
# plotting.plot_dependency_sizes(
#     'right_1_totright_2', 'right_2_totright_2',
#     'MAL',
#     all_langs_average_sizes_filtered,
#     lambda x: True,
#     langNames, langnameGroup,
#     palette=group_to_color
# )

# producing loads of plots

this takes > 12 minutes

In [4]:
# Improved Cell: Separate VO/OV plots and reduce verbosity
import pandas as pd
import plotting
from importlib import reload
from tqdm.notebook import tqdm
# Reload plotting to get the latest sns.scatterplot changes and silent mode
reload(plotting)

print("Starting generation of ALL plot subsets...")

# 1. Prepare Data for Filters
# ---------------------------
# Create mapping for Head Initiality & VO Score
if 'head_initiality_score' in vo_hi_df.columns:
    vo_hi_df['head_initiality'] = vo_hi_df['head_initiality_score']

join_col_hi = 'language' if 'language' in vo_hi_df.columns else 'language_code'
lang_to_head_init = vo_hi_df.set_index(join_col_hi)['head_initiality'].to_dict()

# Load vo_type classification (tripartite: VO/OV/NDO)
if 'vo_type' in vo_hi_df.columns:
    lang_to_vo_type = vo_hi_df.set_index(join_col_hi)['vo_type'].to_dict()
else:
    # Fallback: compute from vo_score if vo_type not available
    if 'vo_score' in vo_hi_df.columns:
        lang_to_vo = vo_hi_df.set_index(join_col_hi)['vo_score'].to_dict()
        lang_to_vo_type = {}
        for lang, score in lang_to_vo.items():
            if score is not None and not pd.isna(score):
                if score > 0.666:
                    lang_to_vo_type[lang] = 'VO'
                elif score < 0.333:
                    lang_to_vo_type[lang] = 'OV'
                else:
                    lang_to_vo_type[lang] = 'NDO'
            else:
                lang_to_vo_type[lang] = None
    else:
        lang_to_vo_type = {}
        print("Warning: Neither 'vo_type' nor 'vo_score' found in vo_hi_df. VO/OV/NDO plots will be skipped.")

# Calculate Lists of Languages for Parallel Processing
indo_euro_langs = {l for l in all_langs_average_sizes_filtered if langnameGroup.get(langNames.get(l, l), 'Other') == 'Indo-European'}
non_indo_euro_langs = {l for l in all_langs_average_sizes_filtered if langnameGroup.get(langNames.get(l, l), 'Other') != 'Indo-European'}

head_init_langs = {l for l in all_langs_average_sizes_filtered if lang_to_head_init.get(l, 0) > 0.5}
head_final_langs = {l for l in all_langs_average_sizes_filtered if lang_to_head_init.get(l, 0) <= 0.5}

# Tripartite word order classification
vo_langs = {l for l in all_langs_average_sizes_filtered if lang_to_vo_type.get(l) == 'VO'}
ov_langs = {l for l in all_langs_average_sizes_filtered if lang_to_vo_type.get(l) == 'OV'}
ndo_langs = {l for l in all_langs_average_sizes_filtered if lang_to_vo_type.get(l) == 'NDO'}

# 2. Generate Head-Init vs Factors Plots
# --------------------------------------
print("\n--- Generating Head-Init vs Factors Plots ---")
if not hcs_df.empty:
    merge_col = 'language' if 'language' in hcs_df.columns else 'language_code'
    if merge_col in hcs_df.columns:
        # Use the same join column logic
        join_col_hi = 'language' if 'language' in vo_hi_df.columns else 'language_code'
        head_init_df = vo_hi_df[[join_col_hi, 'head_initiality']].rename(columns={join_col_hi: merge_col})
        
        all_factors_df = pd.merge(hcs_df, head_init_df, on=merge_col, how='inner')
        all_factors_df['group'] = all_factors_df[merge_col].map(lambda x: langnameGroup.get(langNames.get(x, x), 'Other'))
        all_factors_df['language_name'] = all_factors_df[merge_col].map(lambda x: langNames.get(x, x))
        
        plotting.plot_head_initiality_vs_factors(all_factors_df, group_to_color)
    else:
        print("Warning: Merge col mismatch.")
else:
    print("Warning: hcs_factors.csv not available.")

# 3. Generate Scatter Plot Subsets (Batch)
# ----------------------------------------
subsets = [
    ("Head Final Languages", "headFinal-", head_final_langs),
    ("Head Initial Languages", "headInit-", head_init_langs),
    ("Indo-European Languages", "IE-", indo_euro_langs),
    ("Non-Indo-European Languages", "noIE-", non_indo_euro_langs),
    ("VO (Verb-Object) Languages", "VO-", vo_langs),
    ("OV (Object-Verb) Languages", "OV-", ov_langs),
    ("NDO (No Dominant Order) Languages", "NDO-", ndo_langs)
]

print("\nGenerating scatter plots for subsets...")
# Use tqdm for the overall progress
pbar = tqdm(subsets, desc="Subsets")
for name, prefix, lang_set in pbar:
    pbar.set_postfix_str(f"Processing {name}")
    if not lang_set:
        # print(f"Skipping {name} (no languages found)")
        continue
        
    plotting.plot_all(
        all_langs_average_sizes_filtered, 
        langNames, 
        langnameGroup, 
        filter_lang=lang_set,
        langname_group_or_genus=langnameGroup, 
        folderprefix=prefix, 
        palette=group_to_color, 
        parallel=True
    )

print("\n✅ All plots generated!")

Starting generation of ALL plot subsets...

--- Generating Head-Init vs Factors Plots ---
Generating 9 scatter plots (3 versions per factor)...
Total tasks: 9
Using 80 CPU cores for parallel processing


Generating Head-Init Plots:   0%|          | 0/9 [00:00<?, ?it/s]


✅ Completed 6 scatter plots in plots/head_init_vs_factors/

Generating scatter plots for subsets...


Subsets:   0%|          | 0/7 [00:00<?, ?it/s]

_________________________ plotting all ___________________________ headFinal-
Total plots to generate: 60
Using 79 CPU cores for parallel processing


...

Generating plots:   0%|          | 0/60 [00:00<?, ?it/s]

/bigstorage/kim/ragenv/lib/python3.10/site-packages/statsmodels/robust/robust_linear_model.py:289: ConvergenceWarning: Estimated scale is 0.0 indicating that the most last iteration produced a perfect fit of the weighted data.
  warnings.warn('Estimated scale is 0.0 indicating that the most'
/bigstorage/kim/ragenv/lib/python3.10/site-packages/statsmodels/robust/robust_linear_model.py:289: ConvergenceWarning: Estimated scale is 0.0 indicating that the most last iteration produced a perfect fit of the weighted data.
  warnings.warn('Estimated scale is 0.0 indicating that the most'
/bigstorage/kim/ragenv/lib/python3.10/site-packages/statsmodels/robust/robust_linear_model.py:289: ConvergenceWarning: Estimated scale is 0.0 indicating that the most last iteration produced a perfect fit of the weighted data.
  warnings.warn('Estimated scale is 0.0 indicating that the most'
/bigstorage/kim/ragenv/lib/python3.10/site-packages/statsmodels/robust/robust_linear_model.py:289: ConvergenceWarning: Es

\n✅ Completed 60 plots
_________________________ plotting all ___________________________ NDO-
Total plots to generate: 60
Using 79 CPU cores for parallel processing


Generating plots:   0%|          | 0/60 [00:00<?, ?it/s]

\n✅ Completed 60 plots

✅ All plots generated!


# creation of all plots

this takes nearly 8 minutes

In [ ]:
import pandas as pd
import plotting
from importlib import reload
# Reload plotting to get the latest sns.scatterplot changes
reload(plotting)

print("Starting generation of ALL plot subsets...")

# 1. Prepare Data for Filters
# ---------------------------
# Create mapping for Head Initiality
# Note: 'vo_hi_df' should be loaded. If column is 'head_initiality_score', rename for consistency if needed.
if 'head_initiality_score' in vo_hi_df.columns:
    vo_hi_df['head_initiality'] = vo_hi_df['head_initiality_score']

# Create dictionary mapping language code to head initiality score
lang_to_head_init = vo_hi_df.set_index('language')['head_initiality'].to_dict()

# Load vo_type classification (tripartite: VO/OV/NDO)
if 'vo_type' in vo_hi_df.columns:
    lang_to_vo_type = vo_hi_df.set_index('language')['vo_type'].to_dict()
else:
    # Fallback: compute from vo_score if vo_type not available
    if 'vo_score' in vo_hi_df.columns:
        lang_to_vo = vo_hi_df.set_index('language')['vo_score'].to_dict()
        lang_to_vo_type = {}
        for lang, score in lang_to_vo.items():
            if score is not None and not pd.isna(score):
                if score > 0.666:
                    lang_to_vo_type[lang] = 'VO'
                elif score < 0.333:
                    lang_to_vo_type[lang] = 'OV'
                else:
                    lang_to_vo_type[lang] = 'NDO'
            else:
                lang_to_vo_type[lang] = None
    else:
        lang_to_vo_type = {}
        print("Warning: Neither 'vo_type' nor 'vo_score' found in vo_hi_df. VO/OV/NDO plots will be skipped.")

# Calculate Lists of Languages for Parallel Processing (Pickle-safe)
# Lambda functions cannot be pickled, so we pre-calculate the lists of languages.

indo_euro_langs = {l for l in all_langs_average_sizes_filtered if langnameGroup.get(langNames.get(l, l), 'Other') == 'Indo-European'}
non_indo_euro_langs = {l for l in all_langs_average_sizes_filtered if langnameGroup.get(langNames.get(l, l), 'Other') != 'Indo-European'}
head_init_langs = {l for l in all_langs_average_sizes_filtered if lang_to_head_init.get(l, 0) > 0.5}
head_final_langs = {l for l in all_langs_average_sizes_filtered if lang_to_head_init.get(l, 0) <= 0.5}

# Tripartite word order classification
vo_langs = {l for l in all_langs_average_sizes_filtered if lang_to_vo_type.get(l) == 'VO'}
ov_langs = {l for l in all_langs_average_sizes_filtered if lang_to_vo_type.get(l) == 'OV'}
ndo_langs = {l for l in all_langs_average_sizes_filtered if lang_to_vo_type.get(l) == 'NDO'}


# 2. Generate Head-Init vs Factors Plots
# --------------------------------------
print("\n--- Generating Head-Init vs Factors Plots ---")
    ("Non-Indo-European Languages", "noIE-", non_indo_euro_langs),
    ("VO (Verb-Object) Languages", "VO-", vo_langs),
    ("OV (Object-Verb) Languages", "OV-", ov_langs),
    ("NDO (No Dominant Order) Languages", "NDO-", ndo_langs)
]

for name, prefix, lang_set in subsets:
    print(f"\n--- Generating {name} Plots ({prefix}) ---")
    plotting.plot_all(
        all_langs_average_sizes_filtered, 
        langNames, 
        langnameGroup, 
        filter_lang=lang_set,  # Passing a SET of languages instead of a function
        langname_group_or_genus=langnameGroup, 
        folderprefix=prefix, 
        palette=group_to_color, 
        parallel=True



print("\n✅ All plots generated!")    )else:
    print("Warning: hcs_factors.csv not available.")


# 3. Generate Scatter Plot Subsets (Batch)
# ----------------------------------------
subsets = [
    ("Head Final Languages", "headFinal-", head_final_langs),
    ("Head Initial Languages", "headInit-", head_init_langs),
    ("Indo-European Languages", "IE-", indo_euro_langs),
    ("Non-Indo-European Languages", "noIE-", non_indo_euro_langs)
]

for name, prefix, lang_set in subsets:
    print(f"\n--- Generating {name} Plots ({prefix}) ---")
    plotting.plot_all(
        all_langs_average_sizes_filtered, 
        langNames, 
        langnameGroup, 
        filter_lang=lang_set,  # Passing a SET of languages instead of a function
        langname_group_or_genus=langnameGroup, 
        folderprefix=prefix, 
        palette=group_to_color, 
        parallel=True
    )

print("\n✅ All plots generated!")

Starting generation of ALL plot subsets...

--- Generating Head-Init vs Factors Plots ---
Generating 9 scatter plots (3 versions per factor)...
Total tasks: 9
Using 80 CPU cores for parallel processing

✅ Completed 6 scatter plots in plots/head_init_vs_factors/

--- Generating Head Final Languages Plots (headFinal-) ---
_________________________ plotting all ___________________________ headFinal-
Total plots to generate: 60
Using 79 CPU cores for parallel processing


Generating plots:   0%|          | 0/60 [00:00<?, ?it/s]

✓ Saved plot: plots/headFinal-scatters/right tot 5 pos 3 vs pos 4 HCS.png (labels=True)
✓ Saved plot: plots/headFinal-scatters/right pos 4 tot 4 vs pos 5 tot 5 DIAG.png (labels=True)
✓ Saved plot: plots/headFinal-scatters/right pos 2 tot 5 vs tot 4 MAL.png (labels=True)
✓ Saved plot: plots/headFinal-scatters/right pos 4 tot 5 vs tot 4 MAL.png (labels=True)
✓ Saved plot: plots/headFinal-scatters/right pos 2 tot 4 vs tot 3 MAL.png (labels=True)
✓ Saved plot: plots/headFinal-scatters/right pos 3 tot 3 vs pos 4 tot 4 DIAG.png (labels=True)
✓ Saved plot: plots/headFinal-scatters/right pos 1 tot 5 vs tot 4 MAL.png (labels=True)
✓ Saved plot: plots/headFinal-scatters/right tot 5 pos 4 vs pos 5 HCS.png (labels=True)
✓ Saved plot: plots/headFinal-scatters/right pos 1 tot 3 vs pos 2 tot 4 DIAG.png (labels=True)
✓ Saved plot: plots/headFinal-scatters/right tot 3 pos 1 vs pos 2 HCS.png (labels=True)
✓ Saved plot: plots/headFinal-scatters/right pos 1 tot 2 vs pos 2 tot 3 DIAG.png (labels=True)
✓ Sa

Generating plots:   0%|          | 0/60 [00:00<?, ?it/s]

✓ Saved plot: plots/headInit-scatters/right pos 4 tot 4 vs pos 5 tot 5 DIAG.png (labels=True)
✓ Saved plot: plots/headInit-scatters/right pos 4 tot 5 vs tot 4 MAL.png (labels=True)
✓ Saved plot: plots/headInit-scatters/right tot 4 pos 3 vs pos 4 HCS.png (labels=True)
✓ Saved plot: plots/headInit-scatters/left tot 4 pos 2 vs pos 3 HCS.png (labels=True)
✓ Saved plot: plots/headInit-scatters/left pos 2 tot 5 vs tot 4 MAL.png (labels=True)
✓ Saved plot: plots/headInit-scatters/right pos 3 tot 4 vs tot 3 MAL.png (labels=True)
✓ Saved plot: plots/headInit-scatters/left tot 5 pos 3 vs pos 4 HCS.png (labels=True)
✓ Saved plot: plots/headInit-scatters/right tot 5 pos 4 vs pos 5 HCS.png (labels=True)
✓ Saved plot: plots/headInit-scatters/right tot 3 pos 2 vs pos 3 HCS.png (labels=True)
✓ Saved plot: plots/headInit-scatters/left tot 3 pos 1 vs pos 2 HCS.png (labels=True)
✓ Saved plot: plots/headInit-scatters/left pos 3 tot 5 vs tot 4 MAL.png (labels=True)
✓ Saved plot: plots/headInit-scatters/rig

Generating plots:   0%|          | 0/60 [00:00<?, ?it/s]

✓ Saved plot: plots/IE-scatters/right tot 5 pos 2 vs pos 3 HCS.png (labels=True)
✓ Saved plot: plots/IE-scatters/right pos 2 tot 5 vs tot 4 MAL.png (labels=True)
✓ Saved plot: plots/IE-scatters/left tot 5 pos 2 vs pos 3 HCS.png (labels=True)
✓ Saved plot: plots/IE-scatters/right pos 3 tot 5 vs tot 4 MAL.png (labels=True)
✓ Saved plot: plots/IE-scatters/left pos 1 tot 5 vs tot 4 MAL.png (labels=True)
✓ Saved plot: plots/IE-scatters/right pos 1 tot 3 vs pos 2 tot 4 DIAG.png (labels=True)
✓ Saved plot: plots/IE-scatters/right pos 4 tot 5 vs tot 4 MAL.png (labels=True)
✓ Saved plot: plots/IE-scatters/right pos 2 tot 2 vs pos 3 tot 3 DIAG.png (labels=True)
✓ Saved plot: plots/IE-scatters/right tot 4 pos 1 vs pos 2 HCS.png (labels=True)
✓ Saved plot: plots/IE-scatters/left pos 1 tot 4 vs pos 2 tot 5 DIAG.png (labels=True)
✓ Saved plot: plots/IE-scatters/right pos 1 tot 4 vs pos 2 tot 5 DIAG.png (labels=True)
✓ Saved plot: plots/IE-scatters/left tot 5 pos 1 vs pos 2 HCS.png (labels=True)
✓ Sa

Generating plots:   0%|          | 0/60 [00:00<?, ?it/s]

✓ Saved plot: plots/noIE-scatters/right pos 4 tot 5 vs tot 4 MAL.png (labels=True)
✓ Saved plot: plots/noIE-scatters/right pos 4 tot 4 vs pos 5 tot 5 DIAG.png (labels=True)
✓ Saved plot: plots/noIE-scatters/right pos 1 tot 4 vs pos 2 tot 5 DIAG.png (labels=True)
✓ Saved plot: plots/noIE-scatters/right tot 5 pos 1 vs pos 2 HCS.png (labels=True)
✓ Saved plot: plots/noIE-scatters/right tot 5 pos 2 vs pos 3 HCS.png (labels=True)
✓ Saved plot: plots/noIE-scatters/right pos 2 tot 5 vs tot 4 MAL.png (labels=True)
✓ Saved plot: plots/noIE-scatters/right pos 1 tot 5 vs tot 4 MAL.png (labels=True)
✓ Saved plot: plots/noIE-scatters/left tot 5 pos 4 vs pos 5 HCS.png (labels=True)
✓ Saved plot: plots/noIE-scatters/left pos 2 tot 4 vs pos 3 tot 5 DIAG.png (labels=True)
✓ Saved plot: plots/noIE-scatters/left pos 2 tot 5 vs tot 4 MAL.png (labels=True)
✓ Saved plot: plots/noIE-scatters/left pos 3 tot 4 vs pos 4 tot 5 DIAG.png (labels=True)
✓ Saved plot: plots/noIE-scatters/left pos 4 tot 5 vs tot 4 MAL